In [1]:
import ratinabox
from ratinabox import Environment, Agent
from ratinabox.Neurons import PlaceCells
import numpy as np
import shapely
import matplotlib.pyplot as plt
import pickle

In [2]:
N_PLACE_CELLS = 20 # Number of place cells per arm


In [3]:
# --- Figure 8 maze
Env = Environment(params={
    'aspect':1,
    'holes' : [
        [[0.1,0.1],[0.45,0.1],[0.45,0.9],[0.1,0.9]],
        [[0.55,0.1],[0.9,0.1],[0.9,0.9],[0.55,0.9]]  
    ]
})



# --- Trajectory to alternate between two arms of the maze
left_trajectory = shapely.geometry.LineString([(0.5, 0.05), (0.5, 0.95), (0.05, 0.95), (0.05, 0.05), (0.5, 0.05)])
right_trajectory = shapely.geometry.LineString([(0.5, 0.05), (0.5, 0.95), (0.95, 0.95), (0.95, 0.05), (0.5, 0.05)])

def linearized_pos_to_maze(alpha, arm):
    if arm==0: # Left arm
        points = left_trajectory.interpolate(alpha, normalized=True)
    else: # Right arm
        points = right_trajectory.interpolate(alpha, normalized=True)
    xs = [p.x for p in points]
    ys = [p.y for p in points]
    return np.stack([xs, ys], axis=1)


Ag = Agent(Env)
alpha = np.linspace(0, 1, 1000)
points = np.concatenate([
    linearized_pos_to_maze(alpha, 0)[:-1],
    linearized_pos_to_maze(alpha, 1)[:-1]
])

T = 20 # in seconds how much time to spend in each arm
times = np.linspace(0, 2*T, len(points))
Ag.import_trajectory(
    times=times,
    positions=points,
    interpolate=True
)

# --- Place cells for left and right arms
alpha = np.linspace(0, 1, N_PLACE_CELLS) 
place_cell_centres = np.concatenate([
    linearized_pos_to_maze(alpha, 0),
    linearized_pos_to_maze(alpha, 1)
])

PCs = PlaceCells(
    Ag,
    params={
        'place_cell_centres': place_cell_centres,
        'max_fr' : 5,
        'widths': 0.1
    }
)

# --- Run the simulation
for i in range(int(2*T/Ag.dt)): 
    Ag.update()
    PCs.update()


Successfully imported dataset from arrays passed
Total of 40.0 s of data available
'geodesic' wall geometry only supported for enivironments with 1 additional wall (4 bounding walls + 1 additional). Sorry. Using 'line_of_sight' instead.


In [4]:
# --- Generate colors by linearized position within each arm 
alpha = np.linspace(0, 1, N_PLACE_CELLS)
colors = plt.cm.viridis(alpha)

PCs.colors = np.concatenate([colors, colors]) 

In [5]:
with open("ratinabox_output_figure8_viridis.pickle", "wb") as f:
    pickle.dump({"Environment":Env, "Agent":Ag, "PlaceCells":PCs}, f)